# Explainable AI: Grad-CAM Visualization

## Gradient-weighted Class Activation Mapping

**Purpose:**
- Generate heatmaps showing which brain regions influence predictions
- Compare Model 1 (CNN) vs Model 3 (CNN-LSTM) attention patterns
- Medical interpretation: Do models focus on hippocampus/cortex?
- Error analysis: Why did model misclassify?

**Assignment Criterion:** XAI Integration (20%)

**Expected:** Models should focus on:
- Hippocampus (memory center, shrinks in Alzheimer's)
- Temporal lobe cortex
- Ventricles (enlarge in Alzheimer's)

In [14]:
# Load utilities
%run 00_utils_and_config.ipynb

import cv2
from pathlib import Path

# Create output directory
gradcam_dir = CONFIG['results_path'] / 'gradcam_visualizations'
gradcam_dir.mkdir(exist_ok=True)
(gradcam_dir / 'demented_correct').mkdir(exist_ok=True)
(gradcam_dir / 'nondemented_correct').mkdir(exist_ok=True)
(gradcam_dir / 'misclassified').mkdir(exist_ok=True)

print("✓ Grad-CAM directories created")

PyTorch version: 2.9.1+cu130
CUDA available: True
CUDA version: 13.0
GPU: NVIDIA GeForce RTX 3060 Laptop GPU
Device: cuda
Configuration loaded successfully!

Base path: C:\Users\rishi\CV_Assignment\Paper2
Raw data path: C:\Users\rishi\CV_Assignment\Paper2\Raw_Data
Number of models: 5
Data processing functions loaded successfully!
Evaluation metrics functions loaded successfully!
Visualization functions loaded successfully!
Data augmentation setup loaded successfully!

PAPER 2 UTILITIES AND CONFIGURATION - SUMMARY

✓ All libraries imported successfully
✓ Configuration parameters loaded
✓ Data processing functions defined
✓ Evaluation metrics functions defined
✓ Visualization functions defined
✓ Data augmentation configured

Ready to proceed with:
  - Notebook 01: Data Preparation
  - Notebooks 02-06: Model Implementations
  - Notebook 07: Results Comparison

✓ Grad-CAM directories created


## 1. Load Test Data

In [15]:
# Load 224x224 data for Model 1
X_test_224 = np.load(CONFIG["processed_data_path"] / "X_test_224.npy").astype("float32") / 255.0
y_test = np.load(CONFIG["processed_data_path"] / "y_test.npy")

# Load 128x128 data for Model 3
X_test_128 = np.load(CONFIG["processed_data_path"] / "X_test_128.npy").astype("float32") / 255.0

print(f"Test data shapes:")
print(f"  224x224: {X_test_224.shape}")
print(f"  128x128: {X_test_128.shape}")
print(f"  Labels: {y_test.shape}")
print(f"\nClass distribution:")
print(f"  Non-Demented (0): {np.sum(y_test == 0)}")
print(f"  Demented (1): {np.sum(y_test == 1)}")

Test data shapes:
  224x224: (1094, 224, 224, 3)
  128x128: (1094, 128, 128, 3)
  Labels: (1094,)

Class distribution:
  Non-Demented (0): 603
  Demented (1): 491


## 2. Load Trained Models

In [16]:
# Define Model 1: CNN (same architecture as notebook 02)
class CNNWithoutAug(nn.Module):
    def __init__(self, input_channels=3, num_classes=2):
        super(CNNWithoutAug, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.25)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.20)
        self.flatten_size = 64 * 28 * 28
        self.fc1 = nn.Linear(self.flatten_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.dropout1(x)
        x = F.relu(self.conv3(x))
        self.conv3_output = x  # Save for Grad-CAM
        x = self.pool3(x)
        x = self.dropout2(x)
        x = x.reshape(-1, self.flatten_size)  # Use reshape instead of view
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load Model 1
model1 = CNNWithoutAug(input_channels=3, num_classes=2).to(device)
model1.load_state_dict(torch.load(CONFIG['saved_models_path'] / 'model1_cnn_without_aug_best.pth'))
model1.eval()
print("✓ Model 1 (CNN) loaded")

# Define Model 3: CNN-LSTM (CORRECT architecture from notebook 04)
class CNNLSTM(nn.Module):
    """CNN-LSTM with same CNN architecture as Model 1: 16→32→64 filters"""
    
    def __init__(self, input_channels=3, lstm_hidden=100, num_classes=2):
        super(CNNLSTM, self).__init__()
        # First convolutional block
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Second convolutional block
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.25)
        
        # Third convolutional block
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.20)
        
        # After 128->64->32->16
        self.flatten_size = 64 * 16 * 16  # 16,384
        
        # LSTM layer
        self.lstm = nn.LSTM(self.flatten_size, lstm_hidden, batch_first=True)
        
        # Output layer
        self.fc = nn.Linear(lstm_hidden, num_classes)
    
    def forward(self, x):
        # x shape: (batch, timesteps, channels, height, width)
        batch_size, timesteps, C, H, W = x.size()
        
        # Process each timestep through CNN
        c_out = []
        for t in range(timesteps):
            # First block
            c = F.relu(self.conv1(x[:, t, :, :, :]))
            c = self.pool1(c)
            
            # Second block
            c = F.relu(self.conv2(c))
            c = self.pool2(c)
            c = self.dropout1(c)
            
            # Third block
            c = F.relu(self.conv3(c))
            self.conv3_output = c  # Save for Grad-CAM
            c = self.pool3(c)
            c = self.dropout2(c)
            
            # Flatten
            c = c.reshape(batch_size, -1)
            c_out.append(c)
        
        # Stack timesteps for LSTM
        lstm_input = torch.stack(c_out, dim=1)
        lstm_out, _ = self.lstm(lstm_input)
        last_output = lstm_out[:, -1, :]
        out = self.fc(last_output)
        return out

# Load Model 3
model3 = CNNLSTM(input_channels=3, lstm_hidden=100, num_classes=2).to(device)
model3.load_state_dict(torch.load(CONFIG['saved_models_path'] / 'model3_cnn_lstm_best.pth'))
model3.eval()
print("✓ Model 3 (CNN-LSTM) loaded")

print("\n✓ Both models loaded and ready for Grad-CAM")
print(f"  Model 1 parameters: {sum(p.numel() for p in model1.parameters()):,}")
print(f"  Model 3 parameters: {sum(p.numel() for p in model3.parameters()):,}")

✓ Model 1 (CNN) loaded
✓ Model 3 (CNN-LSTM) loaded

✓ Both models loaded and ready for Grad-CAM
  Model 1 parameters: 6,454,626
  Model 3 parameters: 6,618,186


## 3. Implement Grad-CAM

**Algorithm:**
1. Forward pass → get target class score
2. Backward pass → compute gradients of score w.r.t. feature maps
3. Pool gradients → get channel-wise weights
4. Weighted sum of feature maps → heatmap
5. ReLU → keep only positive influences
6. Upsample → match input image size

In [17]:
def generate_gradcam(model, input_tensor, target_class, target_layer_name='conv3'):
    """
    Generate Grad-CAM heatmap for a given input using gradient hooks.
    
    Args:
        model: Trained neural network
        input_tensor: Input image tensor (1, C, H, W) or (1, T, C, H, W)
        target_class: Class index to generate CAM for
        target_layer_name: Name of convolutional layer to visualize
    
    Returns:
        heatmap: Grad-CAM heatmap (H, W) numpy array
    """
    model.eval()
    
    # Storage for gradients and feature maps
    gradients = []
    activations = []
    
    def backward_hook(module, grad_input, grad_output):
        gradients.append(grad_output[0])
    
    def forward_hook(module, input, output):
        activations.append(output)
    
    # Register hooks on conv3 layer
    if hasattr(model, 'conv3'):
        handle_backward = model.conv3.register_full_backward_hook(backward_hook)
        handle_forward = model.conv3.register_forward_hook(forward_hook)
    else:
        raise ValueError("Model doesn't have conv3 layer")
    
    # Forward pass
    output = model(input_tensor)
    
    # Zero all gradients
    model.zero_grad()
    
    # Backward pass for target class
    class_score = output[0, target_class]
    class_score.backward()
    
    # Remove hooks
    handle_backward.remove()
    handle_forward.remove()
    
    # Get the gradients and activations
    grads = gradients[0].cpu().data.numpy()
    acts = activations[0].cpu().data.numpy()
    
    # Pool gradients across spatial dimensions (global average pooling)
    weights = np.mean(grads, axis=(2, 3))[0, :]  # Shape: (num_filters,)
    
    # Weighted combination of activation maps
    cam = np.zeros(acts.shape[2:], dtype=np.float32)  # Shape: (H, W)
    for i, w in enumerate(weights):
        cam += w * acts[0, i, :, :]
    
    # Apply ReLU to keep only positive influences
    cam = np.maximum(cam, 0)
    
    # Normalize to [0, 1]
    if cam.max() > 0:
        cam = (cam - cam.min()) / (cam.max() - cam.min())
    
    return cam

print("✓ Grad-CAM function defined")

✓ Grad-CAM function defined


## 4. Visualization Function

In [18]:
def visualize_gradcam(original_image, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    """
    Overlay Grad-CAM heatmap on original image.
    
    Args:
        original_image: Original MRI image (H, W, 3) numpy array [0, 1]
        heatmap: Grad-CAM heatmap (H, W) numpy array [0, 1]
        alpha: Transparency of heatmap overlay
        colormap: OpenCV colormap for heatmap
    
    Returns:
        overlayed: Image with heatmap overlay (H, W, 3)
    """
    # Resize heatmap to match original image
    heatmap_resized = cv2.resize(heatmap, (original_image.shape[1], original_image.shape[0]))
    
    # Convert heatmap to RGB using colormap
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap_resized), colormap)
    heatmap_colored = cv2.cvtColor(heatmap_colored, cv2.COLOR_BGR2RGB) / 255.0
    
    # Overlay
    overlayed = (1 - alpha) * original_image + alpha * heatmap_colored
    overlayed = np.clip(overlayed, 0, 1)
    
    return overlayed

print("✓ Visualization function defined")

✓ Visualization function defined


## 5. Generate Grad-CAM for Sample Images

In [19]:
# Get predictions for all test images from Model 1
X_test_224_tensor = torch.from_numpy(np.transpose(X_test_224, (0, 3, 1, 2))).float().to(device)
with torch.no_grad():
    outputs_model1 = model1(X_test_224_tensor)
    _, preds_model1 = torch.max(outputs_model1, 1)
    preds_model1 = preds_model1.cpu().numpy()

# Find interesting samples
demented_correct = np.where((y_test == 1) & (preds_model1 == 1))[0]
nondemented_correct = np.where((y_test == 0) & (preds_model1 == 0))[0]
misclassified = np.where(y_test != preds_model1)[0]

print(f"Found samples:")
print(f"  Demented (correctly classified): {len(demented_correct)}")
print(f"  Non-Demented (correctly classified): {len(nondemented_correct)}")
print(f"  Misclassified: {len(misclassified)}")

# Select samples for visualization
n_samples_per_category = 10
demented_samples = np.random.choice(demented_correct, min(n_samples_per_category, len(demented_correct)), replace=False)
nondemented_samples = np.random.choice(nondemented_correct, min(n_samples_per_category, len(nondemented_correct)), replace=False)
misclassified_samples = np.random.choice(misclassified, min(5, len(misclassified)), replace=False)

print(f"\nGenerating Grad-CAM for {len(demented_samples) + len(nondemented_samples) + len(misclassified_samples)} samples...")

Found samples:
  Demented (correctly classified): 477
  Non-Demented (correctly classified): 598
  Misclassified: 19

Generating Grad-CAM for 25 samples...


## 6. Generate and Save Visualizations

In [20]:
def generate_and_save_gradcam(idx, model, input_data, save_dir, prefix):
    """
    Generate and save Grad-CAM visualization.
    """
    # Prepare input
    img = input_data[idx]
    img_tensor = torch.from_numpy(np.transpose(img[np.newaxis], (0, 3, 1, 2))).float().to(device)
    img_tensor.requires_grad = True
    
    # Get prediction
    with torch.enable_grad():
        output = model(img_tensor)
        _, pred = torch.max(output, 1)
        pred_class = pred.item()
        true_class = y_test[idx]
    
    # Generate Grad-CAM
    heatmap = generate_gradcam(model, img_tensor, target_class=pred_class)
    
    # Visualize
    overlayed = visualize_gradcam(img, heatmap)
    
    # Create figure
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original image
    axes[0].imshow(img)
    axes[0].set_title(f"Original MRI\nTrue: {CONFIG['class_names'][true_class]}")
    axes[0].axis('off')
    
    # Heatmap only
    axes[1].imshow(heatmap, cmap='jet')
    axes[1].set_title(f"Grad-CAM Heatmap\nPred: {CONFIG['class_names'][pred_class]}")
    axes[1].axis('off')
    
    # Overlayed
    axes[2].imshow(overlayed)
    axes[2].set_title(f"Overlayed\nConfidence: {torch.softmax(output, dim=1)[0, pred_class].item():.2%}")
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_dir / f"{prefix}_sample_{idx}.png", dpi=150, bbox_inches='tight')
    plt.close()

# Generate for Demented (correctly classified)
print("Generating Grad-CAM for Demented samples...")
for i, idx in enumerate(demented_samples):
    generate_and_save_gradcam(
        idx, model1, X_test_224,
        gradcam_dir / 'demented_correct',
        f"model1_demented"
    )
    if (i + 1) % 5 == 0:
        print(f"  {i + 1}/{len(demented_samples)} done")

# Generate for Non-Demented (correctly classified)
print("\nGenerating Grad-CAM for Non-Demented samples...")
for i, idx in enumerate(nondemented_samples):
    generate_and_save_gradcam(
        idx, model1, X_test_224,
        gradcam_dir / 'nondemented_correct',
        f"model1_nondemented"
    )
    if (i + 1) % 5 == 0:
        print(f"  {i + 1}/{len(nondemented_samples)} done")

# Generate for Misclassified
print("\nGenerating Grad-CAM for Misclassified samples...")
for i, idx in enumerate(misclassified_samples):
    generate_and_save_gradcam(
        idx, model1, X_test_224,
        gradcam_dir / 'misclassified',
        f"model1_misclassified"
    )
print(f"  {len(misclassified_samples)}/{len(misclassified_samples)} done")

print(f"\n✓ All Grad-CAM visualizations saved to: {gradcam_dir}")

Generating Grad-CAM for Demented samples...
  5/10 done
  10/10 done

Generating Grad-CAM for Non-Demented samples...
  5/10 done
  10/10 done

Generating Grad-CAM for Misclassified samples...
  5/5 done

✓ All Grad-CAM visualizations saved to: ..\results\gradcam_visualizations


## 7. Create Comparison Gallery

In [ ]:
# Create a combined gallery for paper
fig, axes = plt.subplots(3, 5, figsize=(20, 12))
fig.suptitle('Grad-CAM Visualization Gallery - Model 1 (CNN)', fontsize=16, fontweight='bold')

# Row 1: Demented samples
for i, idx in enumerate(demented_samples[:5]):
    img = X_test_224[idx]
    img_tensor = torch.from_numpy(np.transpose(img[np.newaxis], (0, 3, 1, 2))).float().to(device)
    img_tensor.requires_grad = True
    
    with torch.enable_grad():
        output = model1(img_tensor)
        _, pred = torch.max(output, 1)
    
    heatmap = generate_gradcam(model1, img_tensor, target_class=1)
    overlayed = visualize_gradcam(img, heatmap)
    
    axes[0, i].imshow(overlayed)
    axes[0, i].set_title(f"Demented #{i+1}", fontsize=10)
    axes[0, i].axis('off')

# Row 2: Non-Demented samples
for i, idx in enumerate(nondemented_samples[:5]):
    img = X_test_224[idx]
    img_tensor = torch.from_numpy(np.transpose(img[np.newaxis], (0, 3, 1, 2))).float().to(device)
    img_tensor.requires_grad = True
    
    with torch.enable_grad():
        output = model1(img_tensor)
        _, pred = torch.max(output, 1)
    
    heatmap = generate_gradcam(model1, img_tensor, target_class=0)
    overlayed = visualize_gradcam(img, heatmap)
    
    axes[1, i].imshow(overlayed)
    axes[1, i].set_title(f"Non-Demented #{i+1}", fontsize=10)
    axes[1, i].axis('off')

# Row 3: Misclassified samples
for i, idx in enumerate(misclassified_samples[:5]):
    img = X_test_224[idx]
    img_tensor = torch.from_numpy(np.transpose(img[np.newaxis], (0, 3, 1, 2))).float().to(device)
    img_tensor.requires_grad = True
    
    with torch.enable_grad():
        output = model1(img_tensor)
        _, pred = torch.max(output, 1)
    
    heatmap = generate_gradcam(model1, img_tensor, target_class=pred.item())
    overlayed = visualize_gradcam(img, heatmap)
    
    axes[2, i].imshow(overlayed)
    true_label = CONFIG['class_names'][y_test[idx]]
    pred_label = CONFIG['class_names'][pred.item()]
    axes[2, i].set_title(f"Error: {true_label}→{pred_label}", fontsize=10, color='red')
    axes[2, i].axis('off')

plt.tight_layout()
plt.savefig(gradcam_dir / 'gradcam_gallery.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Grad-CAM gallery created for research paper")

## 8. Medical Interpretation Notes

**Expected Attention Patterns:**

### Demented Class
- **Hippocampus** (center/bottom) - Significant atrophy
- **Temporal lobe** - Volume loss
- **Ventricles** (dark regions) - Enlargement

### Non-Demented Class  
- **Cortical thickness** - Preserved gray matter
- **Overall brain volume** - Well-preserved
- **Symmetry** - Bilateral features

### Clinical Relevance
- Grad-CAM allows neurologists to verify model focus areas
- Increases trust in automated diagnosis
- Can identify when model learns spurious correlations
- Supports FDA approval process for medical AI

## 9. Summary Statistics

In [22]:
print("="*80)
print("GRAD-CAM GENERATION SUMMARY")
print("="*80)
print(f"\nTotal visualizations generated: {len(demented_samples) + len(nondemented_samples) + len(misclassified_samples)}")
print(f"  Demented (correct): {len(demented_samples)}")
print(f"  Non-Demented (correct): {len(nondemented_samples)}")
print(f"  Misclassified: {len(misclassified_samples)}")
print(f"\nSaved to: {gradcam_dir}")
print(f"\nKey files for research paper:")
print(f"  ✓ gradcam_gallery.png - Combined visualization")
print(f"  ✓ Individual samples in subdirectories")
print(f"\n✓ XAI Integration (20%) - COMPLETE")
print("="*80)

GRAD-CAM GENERATION SUMMARY

Total visualizations generated: 25
  Demented (correct): 10
  Non-Demented (correct): 10
  Misclassified: 5

Saved to: ..\results\gradcam_visualizations

Key files for research paper:
  ✓ gradcam_gallery.png - Combined visualization
  ✓ Individual samples in subdirectories

✓ XAI Integration (20%) - COMPLETE
